# MGL869 - Projet personnel

*MGL869 ETS Montreal - Production engineering*

## Abstract

## Authors
- **William PHAN**

---

## Part 1 : Collecte des données

In [ ]:
from Jira import jira_download
from pandas import Index
from numpy import ndarray


### 1.1 - Téléchargement des données Jira

Nous téléchargeons les données si elles ne sont pas déjà présentes dans le dossier de données.

Renvoie le dataframe des données.

Le filtre de requête peut être défini dans le fichier config.ini.

In [ ]:
jira_dataframe = jira_download()

### 1.2 - Nettoyer les données Jira en utilisant pandas

Auparavant, nous avons téléchargé toutes les données de Jira. Maintenant, nous allons nettoyer les données en utilisant pandas. Nous allons conserver seulement certaines colonnes et combiner certaines colonnes.

In [ ]:
keep: [str] = ['Issue key', 'Status', 'Resolution', 'Created', 'Fix Versions Combined', 'Affects Versions Combined']

In [ ]:
affects_version_columns: [str] = [col for col in jira_dataframe.columns if col.startswith('Affects Version/s')]
jira_dataframe['Affects Versions Combined'] = jira_dataframe[affects_version_columns].apply(
    lambda x: ', '.join(x.dropna().astype(str)), axis=1
)

In [ ]:
# Combine the versions into a single column
fix_version_columns: [str] = [col for col in jira_dataframe.columns if col.startswith('Fix Version/s')]

jira_dataframe['Fix Versions Combined'] = jira_dataframe[fix_version_columns].apply(
    lambda x: ', '.join(x.dropna().astype(str)), axis=1
)
jira_dataframe = jira_dataframe.loc[:, keep]

In [ ]:
# Identify columns whose names contain the string 'Issue key'
issue_key_columns: Index = jira_dataframe.columns[jira_dataframe.columns.str.contains('Issue key')]
# Extract the values from these columns as a NumPy array
issue_key_values: ndarray = jira_dataframe[issue_key_columns].values
# Flatten the array to create a one-dimensional list of all 'Issue key' values
flattened_issue_keys: ndarray = issue_key_values.flatten()
# Convert the list into a set to remove duplicates
ids: set = set(flattened_issue_keys)

---


## Part 2 : Analyse du répo git


In [ ]:
from Hive import git_download, commit_analysis, update_commit_dataframe, filter_versions_by_min
from git import Repo, Tag
from pandas import DataFrame
from configparser import ConfigParser
from re import compile
from packaging import version  

### 2.1 - Clonage du répo

In [ ]:
repo: Repo = git_download()

In [ ]:
all_couples = commit_analysis(ids)

### 2.2 - Filtrage des données et couples

In [ ]:
commit_dataframe: DataFrame = DataFrame(all_couples, columns=["Issue key", "File", "Commit"])

In [ ]:
# Languages without whitespaces
config: ConfigParser = ConfigParser()
config.read("config.ini")
languages: [str] = config["GENERAL"]["Languages"].split(",")
languages: [str] = [lang.strip() for lang in languages]
commit_dataframe: DataFrame = commit_dataframe[commit_dataframe['File'].str.endswith(tuple(languages))]

In [ ]:
couples = update_commit_dataframe(commit_dataframe, jira_dataframe)
couples
filtered_couples = couples[couples['Version Affected'].str.contains('2.3.9', na=False)]
filtered_couples

### 2.3 - Collecte des versions filtrées

In [ ]:
releases_regex: [str] = config["GIT"]["ReleasesRegex"].split(",")
tags: Tag = repo.tags
versions: dict = {tag.name: tag.commit for tag in tags}
releases_regex: [str] = [regex.strip() for regex in releases_regex]
releases_regex = [compile(regex) for regex in releases_regex]

In [ ]:
filtered_versions = filter_versions_by_min(versions, releases_regex, "2.0.0")
filtered_versions

In [ ]:
from packaging.version import Version

sorted_versions = dict(
    sorted(filtered_versions.items(), key=lambda item: Version(item[0]), reverse=True)
)

sorted_versions

## Part 3. - Analyse des métriques statiques via Understand

In [ ]:
from Understand.commands import und_create_command, und_purge_command
from Understand.metrics import metrics
from Understand.label import label_all_metrics
from os import path
from Understand import merge_static_metrics
from Understand.enrich import enrich_metrics
from Understand.update import merge_all_metrics

### 3.1 - Création du projet Understand

In [ ]:
hive_git_directory: str = config["GIT"]["HiveGitDirectory"]
data_directory: str = config["GENERAL"]["DataDirectory"]
understand_project_name : str = config["UNDERSTAND"]["UnderstandProjectName"]

understand_project_path : str = path.join(data_directory, hive_git_directory, understand_project_name)

if not path.exists(understand_project_path):
    und_create_command()

In [ ]:
und_purge_command()

### 3.2 - Extraction des métriques


In [ ]:
metrics(filtered_versions)

### 3.3 - Labélisation


In [ ]:
label_all_metrics(couples)

In [ ]:
enrich_metrics(couples)

In [ ]:
v = [
    "2.0.0", "2.0.1", "2.1.0", "2.1.1", "2.2.0", "2.3.0", "2.3.1", "2.3.2",
    "2.3.3", "2.3.4", "2.3.5", "2.3.6", "2.3.7", "2.3.8", "2.3.9", "2.3.10",
    "3.0.0", "3.1.0", "3.1.1", "3.1.2", "3.1.3", "4.0.0", "4.0.1"
]
merge_all_metrics(v)

In [ ]:
merge_static_metrics()

# Part 4. - Entraînement du modèle

In [ ]:
import os
from configparser import ConfigParser
from AI import plot_feature_importance_rf, plot_shap_summary,plot_shap_with_others, evaluate_model, train_model, load_and_prepare_data, load_config
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt


In [ ]:
config: ConfigParser = ConfigParser()
config.read("config.ini")

### 4.1 - Evaluation of RF and LR models for all versions

Dans cette partie, il est important de souligner que le modèle construit se base sur les données du csv augmentée contenant le résultat de TOUTES les versions taguées depuis la 2.0.0. Le fichier se trouve dans le dossier src/Output/temp_static_metrics_output

In [ ]:
data_directory = config["GENERAL"]["DataDirectory"]
output_dir = config["UNDERSTAND"]["FullStaticMetricsOutputDirectory"]
file_name = config["UNDERSTAND"]["MergedStaticMetricsFileName"]
file_path = os.path.join(data_directory, output_dir, file_name)

In [ ]:
# Retrieve configuration settings
config_section = "VERSION_ALL_LAB"
param = load_config(config_section)

model_instance_lr = LogisticRegression(max_iter=5000, class_weight='balanced')
print("Running pipeline with the model: Logistic Regression")
X_train_lr, X_test_lr, y_train_lr, y_test_lr = load_and_prepare_data(file_path,param)
trained_model_lr = train_model(model_instance_lr, X_train_lr, y_train_lr)
metrics_lr = evaluate_model(trained_model_lr, X_test_lr, y_test_lr) 

# Random Forest
model_instance_rf = RandomForestClassifier(class_weight='balanced')
X_train_rf, X_test_rf, y_train_rf, y_test_rf = load_and_prepare_data(file_path,param)
print("Running pipeline with the model: Random Forest")
trained_model_rf = train_model(model_instance_rf, X_train_rf, y_train_rf)
metrics_rf = evaluate_model(trained_model_rf, X_test_rf, y_test_rf)




In [ ]:
metrics_lr

In [ ]:
metrics_rf

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(metrics_lr["FPR"], metrics_lr["TPR"],
         label=f"Logistic Regression (AUC = {metrics_lr['AUC']:.2f})")
plt.plot(metrics_rf["FPR"], metrics_rf["TPR"],
         label=f"Random Forest (AUC = {metrics_rf['AUC']:.2f})")
plt.plot([0, 1], [0, 1], 'k--', label="Random Classifier")
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for All Versions')
plt.legend()
plt.grid()
plt.show()


#### Analyse des métriques :
AUC (aire sous la courbe ROC):
L’AUC de Random Forest (0.96) est largement supérieure à celle de Logistic Regression (0.64). Cela montre que Random Forest est bien meilleur pour séparer les classes (positifs vs négatifs).

Précision :
Random Forest a une précision de 96 % contre 63 % pour Logistic Regression. Cela signifie que Random Forest fait beaucoup moins d’erreurs quand il prédit une classe positive.

Rappel (Recall) :
Avec un rappel de 91 %, Random Forest détecte presque tous les vrais positifs, alors que Logistic Regression n’en détecte que 46 %.

#### Conclusion :
Le modèle Random Forest est clairement le plus performant. Voici pourquoi :

Il a un AUC proche de 1, ce qui montre qu'il distingue très bien les classes positives et négatives.
Il est très précis (96 %) dans ses prédictions.
Il détecte la grande majorité des positifs réels grâce à son rappel élevé (91 %).

### 4.2 Étude de l'importance des métriques appliqué au modèle de régression linéaire (LR)

In [ ]:
plot_shap_summary(trained_model=trained_model_lr, X_train=X_train_lr, X_test=X_test_lr, top_n=10)

#### Analyse : CountSemicolon 

Rouge à droite (valeurs élevées) : Les valeurs élevées de CountSemicolon (points rouges) tendent à être à droite (SHAP positif). Cela indique qu'un grand nombre de points-virgules dans un fichier augmente la probabilité qu'il contienne un bogue.
Bleu (valeurs faibles) : Les faibles valeurs de CountSemicolon (points bleus) sont centrées autour de zéro. Cela signifie que lorsque CountSemicolon est faible, cette variable a peu d'influence sur la prédiction.

#### Analyse : CountStmtExe
Rouge à gauche (SHAP négatif) :

Les points rouges indiquent des valeurs élevées de CountStmtExe.
Ces points rouges sont majoritairement situés à gauche (valeurs SHAP négatives). Cela signifie que plus la valeur de CountStmtExe est élevée, plus elle diminue la probabilité qu'un fichier contienne un bug.
Impact sur les prédictions :

Grand nombre de points rouges à gauche :
Cela montre que beaucoup de fichiers avec des valeurs élevées de CountStmtExe (nombre d'instructions exécutables) réduisent la probabilité prédite d’un bug.
Cela peut indiquer que des fichiers contenant un grand nombre d’instructions exécutables (potentiellement bien organisées ou bien testées) sont moins susceptibles de contenir des bugs.

In [ ]:
plot_shap_with_others(trained_model=trained_model_lr, X_train=X_train_lr, X_test=X_test_lr, top_n=30)


Cette vue résume uniquement l’importance globale (moyenne) absolue, sans afficher les relations locales ni les distributions.

### 4.3 Étude de l'importance des métriques appliqué au modèle de forêt aléatoire (RF)

In [ ]:
plot_feature_importance_rf(trained_model_rf=trained_model_rf, feature_columns=X_train_rf.columns, top_n=30)

### Analyse des résultats et suggestions d'actions pour éviter les bogues

À partir des graphiques montrant les **importances des variables** et les **valeurs SHAP** dans Random Forest uniquement (le meilldeur modèle), nous pouvons identifier les métriques ayant le plus d'impact sur la probabilité de présence d'un bogue. Ces analyses permettent de proposer des actions concrètes pour améliorer la qualité du code et éviter des bogues.

---

#### Actions proposées :

1. **Réduire le nombre de déclarations de variables dans les classes (`CountDeclClassVariable`)**
   - **Observation :** `CountDeclClassVariable` est la métrique la plus influente. Un grand nombre de variables déclarées dans une classe peut rendre le code complexe, difficile à maintenir, et sujet à des erreurs.
   - **Action :**
     - Encourager la modularisation des classes en utilisant des sous-classes ou des objets pour encapsuler les variables liées.
     - Limiter le nombre de variables par classe avec des revues de code automatiques.
   - **Justification :**
     - Une classe plus concise facilite la lisibilité et diminue les risques d’introduction de bogues liés à des dépendances complexes.



2. **Optimiser la gestion des classes de base (`CountClassBase`)**
   - **Observation :** Le nombre de classes de base est fortement corrélé à la probabilité de bogues. Cela peut indiquer des hiérarchies de classes complexes ou mal conçues.
   - **Action :**
     - Simplifier les hiérarchies de classes en limitant la profondeur d’héritage.
     - Appliquer des principes de conception comme **Composition over Inheritance** pour réduire la dépendance à de multiples classes de base.
   - **Justification :**
     - Des hiérarchies simplifiées favorisent une meilleure lisibilité et limitent les erreurs dues à des relations mal comprises entre classes.



3. **Réduire la complexité des entrées (`CountInput`)**
   - **Observation :** Un nombre élevé d’entrées peut indiquer une complexité dans la gestion des données ou une forte dépendance à des sources externes.
   - **Action :**
     - Standardiser les formats d’entrée et limiter les dépendances aux données externes non fiables.
     - Ajouter des validations robustes pour éviter les erreurs dues à des données inattendues.
   - **Justification :**
     - Une gestion claire et standardisée des entrées diminue les risques d’erreurs dues à des données mal formées ou non prévues.



4. **Contrôler le nombre total de méthodes déclarées (`CountDeclMethodAll`)**
   - **Observation :** Un grand nombre de méthodes déclarées est corrélé à une complexité accrue, ce qui augmente les risques de bogues.
   - **Action :**
     - Refactoriser les classes pour réduire le nombre de méthodes, par exemple en regroupant des fonctionnalités similaires.
     - Limiter le nombre de méthodes publiques pour réduire l’exposition à des erreurs externes.
   - **Justification :**
     - Réduire le nombre de méthodes facilite la compréhension et diminue les risques d’introduire des erreurs dans des classes complexes. **Attention**, cependant une subdivision trop pauvre du code aura un effet inverse !



5. **Limiter les modifications de couplage des classes (`CountClassCoupledModified`)**
   - **Observation :** Les modifications fréquentes des relations entre classes couplées sont associées à une probabilité accrue de bogues.
   - **Action :**
     - Minimiser les modifications fréquentes des classes couplées en favorisant la stabilité des interfaces.
     - Utiliser des tests automatisés pour valider les modifications dans les dépendances.
   - **Justification :**
     - Réduire les modifications dans des dépendances critiques diminue le risque d’introduire des bogues dans les interactions entre classes.


---

### ... Vers une analyse plus fine
Bien que ces analyses offrent des perspectives globales basées sur toutes les versions combinées, elles ne tiennent pas compte de l'évolution des métriques au fil des versions. Il serait pertinent d'explorer comment ces métriques varient entre les versions pour mieux comprendre l'impact des changements dans le temps. Cela permettra d'identifier des tendances ou des patterns susceptibles de causer des bogues dans les futures versions.


## Part 5. - Limitations du modèle


In [ ]:
from AI import plot_feature_importance_rf, plot_shap_summary,plot_shap_with_others, evaluate_model, train_model, load_and_prepare_data, load_config, train_and_save_models, plot_metrics_evolution
import os, json

In [ ]:
train_and_save_models()

In [ ]:
data_directory = config["GENERAL"]["DataDirectory"]
output_dir = config["UNDERSTAND"]["StaticModelsDirectory"]
file_name = config["MODEL"]["StaticPerformanceMetricsFile"]
file_path = os.path.join(data_directory, output_dir, file_name)

In [ ]:
with open(file_path, "r") as f:
    results = json.load(f)

filtered_results = {version: metrics for version, metrics in results.items()}

plot_metrics_evolution(filtered_results)


## Part 6. - Dynamic Metrics

In [ ]:
from Dynamic import convert_json_to_csv, merge_static_and_dynamic_csv, build_dependencies, display_hierarchy, collect_dynamic_metrics_v2
from Hive import filter_versions_by_min

In [ ]:
all_versions = filter_versions_by_min(versions, releases_regex,'1.0')
version_json = build_dependencies(all_versions)
#display_hierarchy(version_json)
#version_json

In [ ]:
dynamic_metrics = collect_dynamic_metrics_v2(version_json)

In [ ]:
convert_json_to_csv()

In [ ]:
merge_static_and_dynamic_csv()